In [ ]:
import traceback
import socket
from opendatatools import futures
from datetime import datetime, timedelta
from typing import Optional, Sequence, List
from rpc.client import RpcClient
from rpc.utility import INTERVAL_ADJUSTMENT_MAP
from rpc.utility import get_duration, extract_vt_symbol, to_rq_symbol, handle_df, load_json


def get_server_setting():
    setting = load_json("setting.json")
    return setting


def get_downloaded_data():
    data = load_json("daily_update_bars.json")
    return data


def init_client(host: str, port: int, authkey: bytes):
    client = RpcClient(host, port, authkey)
    client.connect()
    return client


def get_update_symbol(client: RpcClient) -> List:
    symbols = client.get_update_symbol()
    print("待更新的合约列表获取成功：")
    print(symbols)
    return symbols


def save_all_data(client: RpcClient, downloaded_data: dict):
    for vt_symbol, value in downloaded_data.items():
        source_interval = value['rq_interval']
        data_dict = value['data']
        client.save_to_database(data_dict, vt_symbol, source_interval)
        print(f"{vt_symbol}合约数据保存成功")
        

setting = get_server_setting()
host_home = '192.168.0.107' if setting['is_at_home'] else setting['host_home']
host_tencent = setting['host_cloud']
port = setting['port']
authkey = setting['authkey'].encode('ascii')
source_interval = "60m"

In [ ]:
# 从服务器获取需要更新的合约列表
def get_symbols_from_server():   
    symbols = set()
    client_home = init_client(host_home, port, authkey)
    if client_home:
        home_symbols = get_update_symbol(client_home)
        symbols.update(set(home_symbols))
        client_home.close()

    client_tencent = init_client(host_tencent, port, authkey)
    if client_tencent:
        tencent_symbols = get_update_symbol(client_tencent)
        symbols.update(set(tencent_symbols))
        client_tencent.close()
        
    print(symbols)
    return symbols

get_symbols_from_server()

In [ ]:
# 将下载完成的K线数据推送到服务器
def push_data_to_server():
    data = get_downloaded_data()
    
    client_home = init_client(host_home, port, authkey)
    if client_home:
        save_all_data(client_home, data)
        client_home.close()
    client_tencent = init_client(host_tencent, port, authkey)
    if client_tencent:
        save_all_data(client_tencent, data)
        client_tencent.close()
        
    return client_home, client_tencent

push_data_to_server()